<a href="https://colab.research.google.com/github/gskumlehn/maltese-or-poodle/blob/main/mobilenet_maltese_or_poodle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Updates database from github repository

In [11]:
!rm -rf maltese-or-poodle
!git clone https://github.com/gskumlehn/maltese-or-poodle.git

Cloning into 'maltese-or-poodle'...
remote: Enumerating objects: 981, done.
remote: Counting objects: 100% (981/981), done.
remote: Compressing objects: 100% (975/975), done.
remote: Total 981 (delta 7), reused 972 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (981/981), 32.11 MiB | 26.47 MiB/s, done.
Resolving deltas: 100% (7/7), done.


Creates two MobileNet models that share the same architecture and pre-trained weights but operate on different input layers:

model: Processes raw/original input data.

augmented_data_model: Processes augmented versions of the input data.

In [12]:
import keras
from keras.applications import MobileNet

model=MobileNet(weights='imagenet', include_top=False)
augmented_data_model=MobileNet(weights='imagenet', include_top=False)

<ipython-input-12-145d36900d23>:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model=MobileNet(weights='imagenet', include_top=False)
<ipython-input-12-145d36900d23>:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  augmented_data_model=MobileNet(weights='imagenet', include_top=False)


Global average pooling is applied to extract compact feature vectors from the outputs of both the original data model and the augmented data model.

In [13]:
from keras.layers import GlobalAveragePooling2D

x=model.output
x=GlobalAveragePooling2D()(x)

augmented_data_x=augmented_data_model.output
augmented_data_x=GlobalAveragePooling2D()(augmented_data_x)

Creates two separate models for processing original and augmented data, with dense layers added for classification

In [14]:
from keras.models import Model
from keras.layers import Dense

x=Dense(50, activation='relu')(x)
preds=Dense(1, activation='sigmoid')(x)

augmented_data_x=Dense(50, activation='relu')(augmented_data_x)
augmented_data_preds=Dense(1, activation='sigmoid')(augmented_data_x)

model=Model(inputs=model.input, outputs=preds)
augmented_data_model=Model(inputs=augmented_data_model.input, outputs=augmented_data_preds)

Counts layers that have already been trained and set only new ones for training

In [15]:
not_trainable_layer_count = len(model.layers) -1

for layer in model.layers[:not_trainable_layer_count]:
    layer.trainable=False
for layer in model.layers[not_trainable_layer_count:]:
    layer.trainable=True

for layer in augmented_data_model.layers[:not_trainable_layer_count]:
    layer.trainable=False
for layer in augmented_data_model.layers[not_trainable_layer_count:]:
    layer.trainable=True

The dataset directory, target image size, and batch size are defined for preparing and processing image data in TensorFlow.

In [16]:
import tensorflow as tf

base_dir = "/content/maltese-or-poodle/classes2"
image_size = (224, 224)
batch_size = 32

The code loads and splits image data from base_dir into training (80%) and validation (20%) datasets, resizing images and batching them. It also retrieves and prints the class names for reference.

In [17]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

class_names = train_dataset.class_names
print("Classes:", class_names)

Found 387 files belonging to 2 classes.
Using 310 files for training.
Found 387 files belonging to 2 classes.
Using 77 files for validation.
Classes: ['pug', 'whippet']


Sets up a pipeline for augmenting and normalizing images in the training dataset and normalizing the test dataset. This improves generalization during training and ensures consistent input scaling.

In [18]:
from keras.layers import RandomFlip, RandomRotation, RandomZoom, Rescaling
from keras.models import Sequential

data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.2),
    RandomZoom(0.2),
])

normalization_layer = Rescaling(1./255)

augmented_train_dataset = train_dataset.map(lambda x, y: (data_augmentation(normalization_layer(x)), y))
augmented_test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

 Optimizes data pipelines by enabling prefetching, allowing data loading and preprocessing to run concurrently with model training or evaluation, improving computational efficiency.

In [19]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)
augmented_train_dataset = augmented_train_dataset.prefetch(buffer_size=AUTOTUNE)
augmented_test_dataset = augmented_test_dataset.prefetch(buffer_size=AUTOTUNE)

 Bompiles both the original and augmented data models with the Adam optimizer, binary cross-entropy loss, and accuracy as the evaluation metric.

In [20]:
from keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])
augmented_data_model.compile(optimizer=Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

Sets epochs

In [21]:
epochs=70

 Trains the models using the training dataset for a specified number of epochs and evaluates it using the validation dataset after each epoch to monitor its performance.

In [ ]:
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=test_dataset
)

Epoch 1/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.4993 - loss: 0.9345 - val_accuracy: 0.4935 - val_loss: 0.9940
Epoch 2/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.4665 - loss: 0.9710 - val_accuracy: 0.4935 - val_loss: 0.9839
Epoch 3/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.4761 - loss: 0.9613 - val_accuracy: 0.4935 - val_loss: 0.9740
Epoch 4/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.4740 - loss: 0.9599 - val_accuracy: 0.4935 - val_loss: 0.9645
Epoch 5/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.4557 - loss: 0.9624 - val_accuracy: 0.4935 - val_loss: 0.9554
Epoch 6/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.4864 - loss: 0.9229 - val_accuracy: 0.4935 - val_loss: 0.9464
Epoch 7/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.4763 - loss: 0.9065 - val_accuracy: 0.4935 - val_loss: 0.9374
Epoch 8/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.4784 - loss: 0.8940 - val_accuracy: 0.4935 - val_loss:

In [ ]:
augmented_data_history = augmented_data_model.fit(
    augmented_train_dataset,
    epochs=epochs,
    validation_data=augmented_test_dataset,
)

Loads test images, preprocesses them, and makes predictions using both the original and augmented data models. It then prints out the predicted class ("poodle" or "maltese") for each test image.

In [ ]:
model.save(f'{class_names[0]}-or-{class_names[1]}-mobilenet.h5')
from google.colab import files
files.download(f'{class_names[0]}-or-{class_names[1]}-mobilenet.h5')

augmented_data_model.save(f'augmented-data-{class_names[0]}-or-{class_names[1]}-mobilenet.h5')
from google.colab import files
files.download(f'augmented-data-{class_names[0]}-or-{class_names[1]}-mobilenet.h5')

In [ ]:
import os
from keras.preprocessing import image
import numpy as np

test_paths = os.listdir("/content/maltese-or-poodle/test")
for path in test_paths:
  test_image = image.load_img(f'/content/maltese-or-poodle/test/{path}', target_size = (224, 224))

  test_image = image.img_to_array(test_image)
  test_image = np.expand_dims(test_image, axis = 0)
  test_image = test_image/255

  result = model.predict(test_image)
  augmented_data_result = augmented_data_model.predict(test_image)

  if result[0][0] < 0.5:
      prediction = class_names[0]
  else:
      prediction = class_names[1]

  if augmented_data_result[0][0] < 0.5:
      augmentedData_prediction = class_names[0]
  else:
      augmentedData_prediction = class_names[0]

  print(f'Prediction: {path} is {prediction}')
  print(f'Augmented data prediction: {path} is {augmentedData_prediction}')

Generates a line graph comparing the training and validation accuracy of two models (one trained with original data and the other with augmented data) across multiple epochs, helping to visualize the performance differences between them.

In [ ]:
import matplotlib.pyplot as plt

val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

augmented_data_val_loss = augmented_data_history.history['val_loss']
augmented_data_val_accuracy = augmented_data_history.history['val_accuracy']

fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(val_loss)
ax.plot(augmented_data_val_loss)
ax.set_title("Validation Loss")
ax.set_xlabel("Epochs")

ax2 = fig.add_subplot(122)
ax2.plot(val_accuracy)
ax2.plot(augmented_data_val_accuracy)
ax2.set_title("Validation Accuracy")
ax2.set_xlabel("Epochs")
ax2.set_ylim(0, 1)

plt.show()


# plt.plot(epochs, val_loss, 'b.', label='Validation Loss')
# pl't.plot(epochs, augmented_data_val_loss, 'r.', label='Augmented Data Validation Loss')

# plt.plot(epochs, val_accuracy, 'b-', label='Validation Accuracy')
# plt.plot(epochs, augmented_data_val_accuracy, 'r-', label='Augmented Data Validation Accuracy')

# plt.title('Model Accuracy Comparison')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()

In [ ]:
plt.clf()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']
augmented_data_val_loss = augmented_data_history.history['val_loss']
augmented_data_val_accuracy = augmented_data_history.history['val_accuracy']

epochs = np.array(range(1, len(accuracy) + 1))  # Convert to NumPy array for mathematical operations

# Logarithmic regression for Training Accuracy
log_fit_acc = np.polyfit(np.log(epochs), accuracy, 1)  # Fit a log-linear model
log_fit_aug_acc = np.polyfit(np.log(epochs), augmented_data_accuracy, 1)

# Logarithmic regression for Validation Accuracy
log_fit_val_acc = np.polyfit(np.log(epochs), val_accuracy, 1)
log_fit_aug_val_acc = np.polyfit(np.log(epochs), augmented_data_val_accuracy, 1)

# Generate fitted values
log_line_acc = log_fit_acc[0] * np.log(epochs) + log_fit_acc[1]
log_line_aug_acc = log_fit_aug_acc[0] * np.log(epochs) + log_fit_aug_acc[1]

log_line_val_acc = log_fit_val_acc[0] * np.log(epochs) + log_fit_val_acc[1]
log_line_aug_val_acc = log_fit_aug_val_acc[0] * np.log(epochs) + log_fit_aug_val_acc[1]

# Plot regression lines
plt.plot(epochs, log_line_acc, 'b:', label='Log Fit: Training Accuracy')
plt.plot(epochs, log_line_aug_acc, 'r:', label='Log Fit: Augmented Training Accuracy')
plt.plot(epochs, log_line_val_acc, 'b-.', label='Log Fit: Validation Accuracy')
plt.plot(epochs, log_line_aug_val_acc, 'r-.', label='Log Fit: Augmented Validation Accuracy')

# Plot details
plt.title('Model Accuracy with Logarithmic Regression')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
